In [26]:
#!pip install preprocessor
!pip install vaderSentiment

     |████████████████████████████████| 5.8MB 3.0MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Using cached https://files.pythonhosted.org/packages/08/82/917eb137b3e85060e4f7c8dc991fccfafb47aa1f6f59fdd63f406159b03e/murmurhash-1.0.5-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/af/6e/0605ffc1c3861cce4d029a9724f302232f993fe9d0dc6c04fb7bc49b7852/preshed-3.0.5-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/86/85/40b8f66c2dd8f4fd9f09d59b22720cffecf1331e788b8a0cab5bafb353d1/plac-1.1.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6c/f9/9a5658e2f56932e41eb264941f9a2cb7f3ce41a80cb36b2af6ab78e2f8af/catalogue-1.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8a/54/115f0c28a61d56674c3a5e05c46d6c3523ad196e1dcd3e2d8b

In [39]:
from elasticsearch import Elasticsearch 

from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy

#to obtain polarity and subjectivity of texts
from textblob import TextBlob
#to obtain the sentiment (pos,neg,neu) values
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
#text cleaning
import spacy 

import string
#import preprocessor as p
import os
import time
import json
import pandas as pd
import numpy as np
import csv
from datetime import datetime
import re
from collections import Counter

In [2]:
# Connect to the elastic cluster
es=Elasticsearch([{'host':'localhost','port':9200}], timeout=30)
print(es)

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>


# Authenticating Twitter API

If you ran into any authentication errors, regenerate your keys and try again

In [14]:
# Twitter credentials
# Obtain them from your twitter developer account
#deleting keys to push to github

consumer_key = ""
consumer_secret = ""
access_key = ""
access_secret = ""

# Pass your twitter credentials to tweepy via its OAuthHandler
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True) #add wait_on_rate_limit to avoid the rate limit error

# Method 1: Extract tweets using tweepy.Cursor. 
This Doesn't work

# Batch Scraping

Due to the limited number of API calls one can make using a basic and free developer account, (~900 calls every 15 minutes before your access is denied) I created a function that extract 2,500 tweets per run once every 15 minutes (I tried to extract 3,00 and above but that got me denied after the second batch). In this function you specify the:
1. search parameter such as key words and hashtags etc.
2. starting date, after which all tweets would be extracted (you can only extract tweets that are not older than the last 7 days)
3. number of tweets to pull per run
4. number of runs - each run happends once every 15 minutes

You may explore the list of metadata from the tweepy.Cursor object in detail (this is the real messy part).


In [43]:
def scraptweets(search_words, date_since, date_until, numTweets, numRuns):
    
    # Define a for-loop to generate tweets at regular intervals
    # We cannot make large API call in one go. Hence, let's try T times
    
    # Define a pandas dataframe to store the date:
    db_tweets = pd.DataFrame(columns = ['username', 'acctdesc', 'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'tweet', 'hashtags']
                                )
    program_start = time.time()
    for i in range(0, numRuns):
        # We will time how long it takes to scrape tweets for each run:
        start_run = time.time()
        
        # Collect tweets using the Cursor object
        # .Cursor() returns an object that you can iterate or loop over to access the data collected.
        # Each item in the iterator has various attributes that you can access to get information about each tweet
        tweets = tweepy.Cursor(api.search, q=search_words, lang="en", since=date_since, tweet_mode='extended').items(numTweets)
# Store these tweets into a python list
        tweet_list = [tweet for tweet in tweets]
# Obtain the following info (methods to call them out):
        # user.screen_name - twitter handle
        # user.description - description of account
        # user.location - where is he tweeting from
        # user.friends_count - no. of other users that user is following (following)
        # user.followers_count - no. of other users who are following this user (followers)
        # user.statuses_count - total tweets by user
        # user.created_at - when the user account was created
        # created_at - when the tweet was created
        # retweet_count - no. of retweets
        # (deprecated) user.favourites_count - probably total no. of tweets that is favourited by user
        # retweeted_status.full_text - full text of the tweet
        # tweet.entities['hashtags'] - hashtags in the tweet
# Begin scraping the tweets individually:
        noTweets = 0
        for tweet in tweet_list:
            #ignore tweets not between date_since and date_until
            if tweet.created_at<date_since or tweet.created_at>date_until:
                continue
            
            #if tweet in valid date range
            # Pull the values
            username = tweet.user.screen_name
            acctdesc = tweet.user.description
            location = tweet.user.location
            following = tweet.user.friends_count
            followers = tweet.user.followers_count
            totaltweets = tweet.user.statuses_count
            usercreatedts = tweet.user.created_at
            tweetcreatedts = tweet.created_at
            retweetcount = tweet.retweet_count
            hashtags = tweet.entities['hashtags']
            try:
                Tweet = tweet.retweeted_status.full_text
            except AttributeError:  # Not a Retweet
                Tweet = tweet.full_text
# Add the 11 variables to the empty list - ith_tweet:
            ith_tweet = [username, acctdesc, location, following, followers, totaltweets,
                         usercreatedts, tweetcreatedts, retweetcount, Tweet, hashtags]
# Append to dataframe - db_tweets
            db_tweets.loc[len(db_tweets)] = ith_tweet
# increase counter - noTweets  
            noTweets += 1
        
        # Run ended:
        end_run = time.time()
        duration_run = round((end_run-start_run)/60, 2)
        
        print('no. of tweets scraped for run {} is {}'.format(i + 1, noTweets))
        print('time take for {} run to complete is {} mins'.format(i+1, duration_run))
        
        #time.sleep(920) #15 minute sleep time
        time.sleep(5) #reducing sleep time to get results faster
        
# Once all runs have completed, save them to a single csv file:
    from datetime import datetime
    
    # Obtain timestamp in a readable format
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
# Define working path and filename
    path = os.getcwd()
    filename = path + '/tweets/' + to_csv_timestamp + '_stock_tweets.csv'
    
    # Store dataframe in csv with creation date timestamp
    db_tweets.to_csv(filename, index = False)
    
    program_end = time.time()
    print('Scraping has completed!')
    print('Total time taken to scrap is {} minutes.'.format(round(program_end - program_start)/60, 2))

In [50]:
#call the scraptweets function with required paramaeters
search_words = 'elon musk OR nytimes OR washingtonpost OR goldmansachs OR #financial news OR stock OR wall street'
date_since = datetime(2020, 12, 1, 0, 0, 0)
date_until = datetime(2020, 12, 11, 0, 0, 0)
numTweets = 1000
numRuns=3

scraptweets(search_words,date_since,date_until,numTweets,numRuns)

#additional info:
#to scrap specific user profiles
#ANJAN, if you ever see this, its creepy I know :P
#tweets = tweepy.Cursor(api.user_timeline, screen_name='anjanragh21').pages()

no. of tweets scraped for run 1 is 0
time take for 1 run to complete is 0.02 mins
no. of tweets scraped for run 2 is 0
time take for 2 run to complete is 0.01 mins
no. of tweets scraped for run 3 is 0
time take for 3 run to complete is 0.01 mins
Scraping has completed!
Total time taken to scrap is 0.2833333333333333 minutes.


# Method 2: Extract tweets using stream
This works like a charm!

In [65]:
from tweepy import API, Stream, OAuthHandler, TweepError

class TweetStreamListener(StreamListener):
    def __init__(self):
        self.count = 0
        self.count_filtered = 0
        self.filter_ratio = 0
        self.db_tweets=pd.DataFrame(columns = ['user_name', 'location', 'friends', 'followers', 
                                               'created_date', 'text_raw'])
        
    # on success
    def on_data(self, data):
        self.count+=1
        # decode json
        dict_data = json.loads(data)
        #print("\n------------------------------> (tweets: %s, filtered: %s, filter-ratio: %s)" \
        #        % (self.count, self.count_filtered, str(round(self.count_filtered/self.count*100,2))+"%"))
        text = dict_data['text']
        
        # get date when tweet was created
        created_date = time.strftime(
                '%Y-%m-%dT%H:%M:%S', time.strptime(dict_data['created_at'], '%a %b %d %H:%M:%S +0000 %Y'))
                                   
        # store dict_data into vars
        user_name = str(dict_data.get("user", {}).get("screen_name"))
        location = str(dict_data.get("user", {}).get("location"))
        language = str(dict_data.get("user", {}).get("lang"))
        friends = int(dict_data.get("user", {}).get("friends_count"))
        followers = int(dict_data.get("user", {}).get("followers_count"))
        statuses = int(dict_data.get("user", {}).get("statuses_count"))
        tweetid = int(dict_data.get("id"))
        text_raw = str(dict_data.get("text"))

        # output twitter data
        print("\n<------------------------------")
        print("Tweet Date: " + created_date)
        print("User Name: " + user_name)
        print("Location: " + location)
        print("Language: " + language)
        print("Friends: " + str(friends))
        print("Followers: " + str(followers))
        print("Statuses: " + str(statuses))
        print("Tweet ID: " + str(tweetid))
        print("Tweet Raw Text: " + text_raw)
        
        # Add the variables to the empty list - ith_tweet:
        ith_tweet = [user_name, location, friends, followers,
                      created_date, text_raw]
        # Append to dataframe - db_tweets or add to elastic search db if needed
        self.db_tweets.loc[len(self.db_tweets)] = ith_tweet
        
        #save the df - operation inefficient as it saves for every tweet
        self.db_tweets.to_csv('tweets/stock_tweets.csv',index=False)
        # increase counter - noTweets  
        self.count+=1
        
        
            
# create instance of the tweepy tweet stream listener
tweetlistener = TweetStreamListener()

# create instance of the tweepy stream
stream = Stream(auth,tweetlistener)

keywords=['elon', 'musk', 'stock', 'stock market', 'wall street', 'new york stock exchange']
stream.filter(track=keywords, languages=['en'])


<------------------------------
Tweet Date: 2020-12-13T23:05:36
User Name: humblegentsnl
Location: Haarlem, Nederland
Language: None
Friends: 1410
Followers: 6604
Statuses: 26632
Tweet ID: 1338258802114695181
Tweet Raw Text: @leeash10 Hi mate no sorry no longer in stock

<------------------------------
Tweet Date: 2020-12-13T23:05:36
User Name: HiTCapital2
Location: None
Language: None
Friends: 0
Followers: 12
Statuses: 21
Tweet ID: 1338258803167412225
Tweet Raw Text: $CM8 we should see it starting to move back up. I can't imagine there'll be too much more stock changing hands at t… https://t.co/oWq0o5txFZ

<------------------------------
Tweet Date: 2020-12-13T23:05:36
User Name: RonenBenReuben
Location: Los Angeles, CA
Language: None
Friends: 57
Followers: 14
Statuses: 1416
Tweet ID: 1338258803154776064
Tweet Raw Text: @Chargers I get the Chargers think firing Lynn is not going to help this season, but it will help you not look like… https://t.co/RFgDSS6Aoo

<-----------------------


<------------------------------
Tweet Date: 2020-12-13T23:05:56
User Name: HedgeBuddies
Location: Inverurie, Aberdeenshire UK
Language: None
Friends: 6488
Followers: 6570
Statuses: 71308
Tweet ID: 1338258883303895041
Tweet Raw Text: RT @Tanyawarren: Hop to it, these guys are in stock! #handmade #frog  #atsocialmedia #UKSOPRO #giftideas https://t.co/uCvQtJ2LSz #etsy #Cra…

<------------------------------
Tweet Date: 2020-12-13T23:05:56
User Name: KM02758589
Location: 🇨🇦
Language: None
Friends: 253
Followers: 63
Statuses: 2354
Tweet ID: 1338258883547189252
Tweet Raw Text: RT @CollinsKellyE: @EmeraldRobinson Shocker. Next up: Nike, NBA, Goodyear, Google, Hollywood studios, medical institutions, Wall Street, Ha…

<------------------------------
Tweet Date: 2020-12-13T23:05:57
User Name: mike_nunes
Location: Massachusetts, USA
Language: None
Friends: 1046
Followers: 662
Statuses: 40463
Tweet ID: 1338258887607279619
Tweet Raw Text: Oh crap! I have money in the stock market dont i...@SNFonNB


<------------------------------
Tweet Date: 2020-12-13T23:06:12
User Name: cutesope
Location: Cardiff, Wales
Language: None
Friends: 455
Followers: 803
Statuses: 49333
Tweet ID: 1338258952191094786
Tweet Raw Text: it's literally impossible to buy a house in Cardiff because they're all owned by landlords...

<------------------------------
Tweet Date: 2020-12-13T23:06:13
User Name: OcsUpdater
Location: Ontario, Canada
Language: None
Friends: 664
Followers: 484
Statuses: 40685
Tweet ID: 1338258955882147842
Tweet Raw Text: No longer in stock at OCS: 

CBD 1:20 Oil by Verse Originals

https://t.co/uak7M8Dtub

<------------------------------
Tweet Date: 2020-12-13T23:06:13
User Name: TiaagoChaves
Location: None
Language: None
Friends: 1718
Followers: 23
Statuses: 342
Tweet ID: 1338258956653895680
Tweet Raw Text: RT @SeanCar79447369: AntiOx is your place to look for answers to boost your immune system and of course refill your Fountain of Life stock…

<------------------------------
Tweet D


<------------------------------
Tweet Date: 2020-12-13T23:06:26
User Name: Panjsheri_
Location: Washington, DC
Language: None
Friends: 477
Followers: 160
Statuses: 4066
Tweet ID: 1338259009565052928
Tweet Raw Text: RT @ajitosu: @AnthonyOhayon A big yes. $SQ is the most “matured” stock in my portfolio. It can easily grow 2x in the next 5 yrs but by then…

<------------------------------
Tweet Date: 2020-12-13T23:06:26
User Name: FreeDem15
Location: None
Language: None
Friends: 58
Followers: 8
Statuses: 8675
Tweet ID: 1338259010886111233
Tweet Raw Text: RT @Hotel3_: I was just informed from more than one reliable source that this weekend would be a very good time to stock up. Two weeks food…

<------------------------------
Tweet Date: 2020-12-13T23:06:28
User Name: Captkenimo
Location: BlueHawaii, USA
Language: None
Friends: 10131
Followers: 10067
Statuses: 18272
Tweet ID: 1338259018121269254
Tweet Raw Text: Retweeted by @LLinWood 👇🏽👇🏽👇🏽

<------------------------------
Tweet Date: 202


<------------------------------
Tweet Date: 2020-12-13T23:06:38
User Name: carsonbc
Location: Lower Sackville, NS
Language: None
Friends: 49
Followers: 239
Statuses: 2704
Tweet ID: 1338259061100392449
Tweet Raw Text: @VicTafur Gruden and his boyfriend Guenther are a joke and a laughing stock of the league.

<------------------------------
Tweet Date: 2020-12-13T23:06:39
User Name: hah0921
Location: Tampa, FL
Language: None
Friends: 4984
Followers: 3018
Statuses: 52446
Tweet ID: 1338259063948316672
Tweet Raw Text: RT @ukexpat19: NEW RULE: Members of Congress SHOULD NOT own stocks!

<------------------------------
Tweet Date: 2020-12-13T23:06:39
User Name: 34NenaSM
Location: United States
Language: None
Friends: 690
Followers: 609
Statuses: 34756
Tweet ID: 1338259064523026432
Tweet Raw Text: RT @crsphilly: 🇺🇸  PLEASE READ THIS  🇺🇸THREAD AND TAKE NOTICE

The time is NEAR and this time is to be used WISELY! 

Get READY and be prep…

<------------------------------
Tweet Date: 2020-12-13T2


<------------------------------
Tweet Date: 2020-12-13T23:06:53
User Name: WHITEFX84
Location: None
Language: None
Friends: 37
Followers: 14
Statuses: 4532
Tweet ID: 1338259122458775552
Tweet Raw Text: RT @Investingcom: ⚠️BREAKING:

*SUNDAY NIGHT FUTURES OPEN HIGHER AS WALL STREET BRACES FOR ANOTHER EVENTFUL WEEK

$DIA $SPY $QQQ $IWM $VIX…

<------------------------------
Tweet Date: 2020-12-13T23:06:53
User Name: emceebeere
Location: None
Language: None
Friends: 3967
Followers: 878
Statuses: 32113
Tweet ID: 1338259125495533568
Tweet Raw Text: RT @karamballes: 3/ "But Sweden"..."learn to live with it".. "GBD"

Sweden schools are shutting early for Christmas to try and prevent what…

<------------------------------
Tweet Date: 2020-12-13T23:06:54
User Name: uwutism
Location: hails ෆ
Language: None
Friends: 743
Followers: 2203
Statuses: 47868
Tweet ID: 1338259126254788611
Tweet Raw Text: @opidied yeee that’s my issue i wna sell my old gpu before new stock comes in so i can sell it befor


<------------------------------
Tweet Date: 2020-12-13T23:07:09
User Name: JackyBancayan
Location: New York, NY
Language: None
Friends: 344
Followers: 315
Statuses: 320
Tweet ID: 1338259193183264768
Tweet Raw Text: RT @washingtonpost: Perspective: The Wall Street Journal column about Jill Biden is worse than you thought https://t.co/5Q3ubkie2c

<------------------------------
Tweet Date: 2020-12-13T23:07:10
User Name: jgordon_23
Location: Aberdeen, Scotland
Language: None
Friends: 4039
Followers: 205
Statuses: 22028
Tweet ID: 1338259197146886145
Tweet Raw Text: RT @davelackie: Bonus "Happy Holidays, Followers" Twitter Giveaway. I'm saying "season's greetings" to everyone by giving away Givenchy Liv…

<------------------------------
Tweet Date: 2020-12-13T23:07:10
User Name: stocks_eagle
Location: Calgary, Alberta
Language: None
Friends: 128
Followers: 126
Statuses: 1330
Tweet ID: 1338259197109080068
Tweet Raw Text: Get ready guys, its green AF 😂😂😂 https://t.co/MlKupLbPOX

<-----------


<------------------------------
Tweet Date: 2020-12-13T23:07:30
User Name: BenRodriguez413
Location: Suffield, Connecticut
Language: None
Friends: 2345
Followers: 504
Statuses: 61237
Tweet ID: 1338259277300031488
Tweet Raw Text: RT @CoriBush: They had the money for endless wars. 

They had the money for Wall Street bailouts. 

They have the money for pandemic surviv…

<------------------------------
Tweet Date: 2020-12-13T23:07:30
User Name: egapnala65
Location: None
Language: None
Friends: 715
Followers: 737
Statuses: 86741
Tweet ID: 1338259277786517504
Tweet Raw Text: RT @CllrBattley: I can't take any more. If I've told the Signage Dept. once not to hire Sid Woodall &amp; Sons I've told them a thousand times.…

<------------------------------
Tweet Date: 2020-12-13T23:07:30
User Name: buchman_matt
Location: Hazleton, PA
Language: None
Friends: 42
Followers: 30
Statuses: 2
Tweet ID: 1338259278638051328
Tweet Raw Text: RT @PAPrepsRivals: Class of 2023 Running Back Stock Risers 
https:

KeyboardInterrupt: 

In [66]:
tweets_df = pd.read_csv('./tweets/stock_tweets.csv')
print(tweets_df)

          user_name            location  friends  followers  \
0     humblegentsnl  Haarlem, Nederland     1410       6604   
1       HiTCapital2                None        0         12   
2    RonenBenReuben     Los Angeles, CA       57         14   
3          Aniras27             Germany      925         28   
4      Squirrel1110                None      137       1530   
..              ...                 ...      ...        ...   
181      egapnala65                None      715        737   
182    buchman_matt        Hazleton, PA       42         30   
183         C010Car                  Uk       11          0   
184       emkatepsu                None      359        120   
185        DarRider                None     3063       2677   

            created_date                                           text_raw  
0    2020-12-13T23:05:36      @leeash10 Hi mate no sorry no longer in stock  
1    2020-12-13T23:05:36  $CM8 we should see it starting to move back up...  
2    2020

# Remove duplicates

In [67]:
#remove duplicate tweets
tweets_df = tweets_df.drop_duplicates(subset='text_raw')
len(tweets_df)

150

# Filter tweets in date range

In [68]:
tweets_df = tweets_df.set_index(['created_date'])
filtered_df = tweets_df.loc['2020-12-11','2020-12-13']
filtered_df

KeyError: '2020-12-11'

# Text Cleaning

In [27]:
#load eng language
nlp = spacy.load('en_core_web_sm')

def normalise(msg): 
    doc = nlp(msg)
    res = []
    
    for token in doc:
        #Removing Stop words and words out of vocabulary
        if token.is_stop or token.is_punct or token.is_space or not(token.is_oov): 
            pass
        #lemmatize words
        #lemmatize example: going and went are converted to go.
        else:
            res.append(token.lemma_.lower())
    
    res = ' '.join(res)        
    return res

In [28]:
tweets_df['tweets_cleaned']=tweets_df['tweet'].apply(normalise)

## Most common words

In [29]:
words_collection = Counter([word for tweet in tweets_df['tweets_cleaned'] for word in tweet.split()])
freq_word_df = pd.DataFrame(words_collection.most_common(15))
freq_word_df.columns = ['frequently_used_word','count']

freq_word_df.style.background_gradient(cmap='YlGnBu', low=0, high=0, axis=0, subset=None)

,frequently_used_word,count
0,musk,584
1,elon,549
2,starship,193
3,texas,182
4,tesla,171
5,move,132
6,prototype,123
7,news,112
8,make,108
9,big,108


# Subjectivity, Polarity and Sentiment of Tweets

In [30]:
#get subjectivity and polarity for tweets
def get_polarity(tweet):
    return TextBlob(tweet).sentiment.polarity

def get_subjectivity(tweet):
    return TextBlob(tweet).sentiment.subjectivity

In [34]:
#polarity ranges from -1 to 1, -1 is negative and 1 is positive
tweets_df['polarity']=tweets_df['tweets_cleaned'].apply(get_polarity)

#subjectivity ranges from 0 to 1. 0 is objective and 1 is subjective
tweets_df['subjectivity']=tweets_df['tweets_cleaned'].apply(get_subjectivity)

tweets_df

,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,tweet,hashtags,tweets_cleaned,polarity,subjectivity
0,flyingwarhobbit,Retired Colonel of Marines; \nCurrently Employ...,"Margaritaville, USA",977,861,2234,2015-08-13 14:11:49,2020-12-10 21:40:21,0,"Immediately After Moving To Texas, Elon Musk A...",[],immediately move texas elon musk announce tesl...,0.000000,0.000000
1,ScottPa95856164,what is wrong with people,land of THE FREE,27,59,1735,2019-09-27 14:26:18,2020-12-10 21:40:03,4381,"Immediately After Moving To Texas, Elon Musk A...",[],immediately move texas elon musk announce tesl...,0.000000,0.000000
2,s0tiKWSQ0jWyqgh,NaN,NaN,34,14,17,2020-12-04 19:46:21,2020-12-10 21:37:33,56,This news is like:\n\n#Russia 🇷🇺 #Moscow to #S...,"[{'text': 'Russia', 'indices': [37, 44]}, {'te...",news like russia 🇷 🇺 moscow stpetersburg germa...,0.000000,0.000000
4,AmberOwens18,HR Leader @AdventHealth – Passionate about exp...,"Fort Worth, Texas USA",4820,2385,63797,2009-09-26 23:47:52,2020-12-10 21:34:32,0,The Bay Area exodus has a new poster boy. Will...,[],bay area exodus new poster boy change dynamic ...,0.068182,0.310606
5,bloodnteeth,"#WarrenDemocrat Smashing racism, corrupt capit...",NaN,1027,489,34688,2019-09-10 03:07:31,2020-12-10 21:33:54,526,"Oh man... a program designed to fight poverty,...",[],oh man program design fight poverty opportunit...,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,ramonferrandis,Bees have taught me that I don't know jack. Sa...,"Green Bank, WV",1530,633,52629,2011-02-04 14:54:58,2020-12-10 01:34:51,8,"Jeff Bezos, Elon Musk and the rest of the peop...",[],jeff bezos elon musk rest people planet killin...,0.000000,0.000000
2477,ronnynga,NaN,NaN,1455,26,4844,2017-07-18 02:50:00,2020-12-10 01:33:12,11,"Elon Musk and Jeff Bezos, the world’s two rich...",[],elon musk jeff bezos world rich man get extra ...,0.187500,0.425000
2487,stem_ai,Beep boop. I’m a simple robot that retweets #S...,"Plymouth, England",2110,2111,55956,2019-05-12 17:42:39,2020-12-10 01:30:55,1,Elon Musk's Starship prototype makes a big imp...,"[{'text': 'stem', 'indices': [64, 69]}, {'text...",elon musk starship prototype make big impact s...,0.000000,0.100000
2491,lightbryantZZ,"Christian, husband, proud step-father, mathema...",NaN,1555,358,151957,2016-06-08 18:56:10,2020-12-10 01:30:05,0,"Musk, Bezos Space Race Gets a Boost From Anti-...",[],musk bezos space race get boost anti poverty t...,0.000000,0.000000


In [32]:
#get the sentiment scores
def get_sentiment(news):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(news)
    return sentiment

In [35]:
compound,pos,neg,neu,sentiments = [],[],[],[],[]
for index,row in tweets_df.iterrows():
    sentiment = get_sentiment(tweets_df['tweets_cleaned'][index])
    pos.append(sentiment['pos'])
    neg.append(sentiment['neg'])
    neu.append(sentiment['neu'])
    #compound.append(sentiment['compound'])
  
    #find the maximum of pos,neg,neu and create target
    index = np.argmax([sentiment['neg'], sentiment['neu'], sentiment['pos']])
    #negative
    if index==0:
        sentiments.append('negative')
    #neutral
    elif index==1:
        sentiments.append('neutral')
    #positive
    else:
        sentiments.append('positive')  

tweets_df['positive']=pos
tweets_df['negative']=neg
tweets_df['neutral']=neu
#df['compound']=compound
tweets_df['sentiment']=sentiments
tweets_df.head()

,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,tweet,hashtags,tweets_cleaned,polarity,subjectivity,positive,negative,neutral,sentiment
0,flyingwarhobbit,Retired Colonel of Marines; \nCurrently Employ...,"Margaritaville, USA",977,861,2234,2015-08-13 14:11:49,2020-12-10 21:40:21,0,"Immediately After Moving To Texas, Elon Musk A...",[],immediately move texas elon musk announce tesl...,0.000000,0.000000,0.000,0.000,1.000,neutral
1,ScottPa95856164,what is wrong with people,land of THE FREE,27,59,1735,2019-09-27 14:26:18,2020-12-10 21:40:03,4381,"Immediately After Moving To Texas, Elon Musk A...",[],immediately move texas elon musk announce tesl...,0.000000,0.000000,0.000,0.000,1.000,neutral
2,s0tiKWSQ0jWyqgh,NaN,NaN,34,14,17,2020-12-04 19:46:21,2020-12-10 21:37:33,56,This news is like:\n\n#Russia 🇷🇺 #Moscow to #S...,"[{'text': 'Russia', 'indices': [37, 44]}, {'te...",news like russia 🇷 🇺 moscow stpetersburg germa...,0.000000,0.000000,0.121,0.036,0.843,neutral
4,AmberOwens18,HR Leader @AdventHealth – Passionate about exp...,"Fort Worth, Texas USA",4820,2385,63797,2009-09-26 23:47:52,2020-12-10 21:34:32,0,The Bay Area exodus has a new poster boy. Will...,[],bay area exodus new poster boy change dynamic ...,0.068182,0.310606,0.245,0.000,0.755,neutral
5,bloodnteeth,"#WarrenDemocrat Smashing racism, corrupt capit...",NaN,1027,489,34688,2019-09-10 03:07:31,2020-12-10 21:33:54,526,"Oh man... a program designed to fight poverty,...",[],oh man program design fight poverty opportunit...,0.000000,0.000000,0.113,0.239,0.648,neutral


In [40]:
# Obtain timestamp in a readable format
timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')

tweets_df.to_csv('tweets_labelled/'+str(timestamp)+'_full_df.csv',index=False,header=True)

# References

https://medium.com/python-in-plain-english/scraping-tweets-with-tweepy-python-59413046e788

https://medium.com/python-in-plain-english/create-a-simple-covid-19-dashboard-with-flask-plotly-altair-chart-js-and-adminlte-a92ef86a3ca8